In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.feature_selection import chi2
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import rand_score

In [ ]:
X = pd.read_csv('X.csv')
y = pd.read_csv('y.csv')
time = pd.read_csv('time.csv')

In [ ]:
X['time'] = time

In [1]:
X

NameError: name 'X' is not defined

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
y_train.value_counts(normalize=True)

In [ ]:
chi2_stats, p_values = chi2(X_train, y_train)
chi2_stats

In [ ]:
good_colums = []
for x, y in zip(chi2_stats,X_train.columns.tolist()):
  if x > 0.2:
    good_colums.append(y)
    
    
X_train_X = X_train[good_colums]
X_test_X = X_test[good_colums]

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score

In [ ]:
# Algorytm PCA nie działa na kolumny binarne, a MCA jedynie zabiera interpretowalność danych

pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),  
    ('scaler', StandardScaler()),               
    ])

In [ ]:
X_train_pipe = pipeline.fit_transform(X_train_X)
X_test_pipe = pipeline.fit_transform(X_test_X)

In [ ]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score

In [ ]:
classifiers = {
    'CatBoost': CatBoostClassifier(),
    'XGBoost': XGBClassifier(),
    'DecisionTree': DecisionTreeClassifier(),
}

param_grids = {
    'DecisionTree': {
        'max_depth': [150, 200],
        'min_samples_split': [5, 10]
    },
    'XGBoost': {
        'n_estimators': [100, 150],
        'max_depth': [6, 9],
        'learning_rate': [1.5, 2],
        'subsample': [0.4, 0.6,]
    },
    'CatBoost': {
        'iterations': [300, 400],
        'depth': [8, 10, 12],
        'learning_rate': [1, 1.5, 2],
    }
}

In [ ]:
best_estimators = {}
for clf_name in classifiers:
    print(f"Grid Search dla {clf_name}")
    grid_search = GridSearchCV(classifiers[clf_name], param_grids[clf_name], cv=5, scoring='recall', n_jobs=-1)
    grid_search.fit(X_train_pipe, y_train)
    best_estimators[clf_name] = grid_search.best_estimator_
    print(f"Najlepsze parametry dla {clf_name}: {grid_search.best_params_}")
    print(f"Najlepszy wynik dla {clf_name}: {grid_search.best_score_}\n")

In [ ]:
DT = DecisionTreeClassifier(max_depth= 200, min_samples_split = 5)
XGBC = XGBClassifier(learning_rate = 2, max_depth = 9, n_estimators = 150, subsample=0.6)
CBC = CatBoostClassifier(depth=8, iterations=300, learning_rate=2)

In [ ]:
DT.fit(X_train_pipe, y_train)
XGBC.fit(X_train_pipe, y_train)
CBC.fit(X_train_pipe, y_train)

In [ ]:
y_pred_DT = DT.predict(X_test_pipe)

print(recall_score(y_test, y_pred_DT))
print(precision_score(y_test, y_pred_DT))
print(confusion_matrix(y_test, y_pred_DT))

In [ ]:
y_pred_XGBC = XGBC.predict(X_test_pipe)

print(recall_score(y_test, y_pred_XGBC))
print(precision_score(y_test, y_pred_XGBC))
print(confusion_matrix(y_test, y_pred_XGBC))

In [ ]:
y_pred_CBC = CBC.predict(X_test_pipe)

print(recall_score(y_test, y_pred_CBC))
print(precision_score(y_test, y_pred_CBC))
print(confusion_matrix(y_test, y_pred_CBC))